In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os, sys
form tqdm import tqdm

In [ ]:
def preprocess_image(image):
    try:
        # load to gray image
        ori = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(ori, cv2.COLOR_BGR2GRAY)
        
        # 用Sobel算子計算x，y方向上的梯度，之後在x方向上減去y方向上的梯度
        # 我們留下具有高水平梯度和低垂直梯度的圖像區域
        gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
        # subtract the y-gradient from the x-gradient
        gradient = cv2.subtract(gradX, gradY)
        gradient = cv2.convertScaleAbs(gradient)

        # blur and the image 去除圖像上的噪聲。
        blurred = cv2.blur(gradient, (9, 9))

        # threshold the image 對模糊圖像二值化
        (_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # perform a series of erosions and dilations 
        closed = cv2.erode(closed, None, iterations=4) # 侵蝕
        closed = cv2.dilate(closed, None, iterations=58) # 膨脹

        # find Contour 
        (cnts,_) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        c = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

        # compute the rotated bounding box of the largest contour
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))

        # find image bounding box
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]

        x1 = min(Xs)
        y1 = min(Ys)     
        x2 = max(Xs)
        y2 = max(Ys)

        if x1 < 0:
            x1 = 0
        if y1 < 0:    
            y1 = 0
        
        hight = y2 - y1
        width = x2 - x1
        crop_img = image[y1:y1+hight, x1:x1+width]

        img_result = tf.image.resize_with_crop_or_pad(crop_img,590,590)

        return img_result
    except IndexError:
        print("list out of range")
        return -1

In [ ]:
if __name__ == "__main__":
    
    folders = ["zone1","zone2","zone3"]
    years = [102,103,104,105,106,107,108,109]
    
    # open folder
    for folder in folders:
        for year in years:
            
            # find all images path
            folderpath = r"./%s/%s"%(folder,year)
            dirs = os.listdir(folderpath)
            
            if not os.path.exists(r"./%s/out_%s" % (folder,year)):
                os.mkdir(r"./%s/out_%s" % (folder,year))
            
            # input images
            for file in dirs:
                if file != ".ipynb_checkpoints":
                    imgpath = folderpath + "/%s"%(file)
                    image = cv2.imread(imgpath)

                    if image is None:
                        print ('Error opening image: ' + imgpath)

                    #transfer image type EagerTensor to numpy.ndarray
                    img_result = preprocess_image(image)
                    img_result = np.array(img_result)

                    # save image
                    cv2.imwrite(folderpath + "_out/%s"%(file), img_result)
                    plt.show()